In [1]:
# I took the rough structure of the file from the example in the lecture and adapted it to my needs
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("fifacalculations") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/06 14:01:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read input data from the fifa dataset (data/clean_fifa_worldcup_matches.csv)
fifa_file = spark.read.csv("../../../data/clean_fifa_worldcup_matches.csv", header=True, sep=",")

In [3]:
# Test if the data is read correctly
fifa_file.show()
#Show the shape of the data
print((fifa_file.count(), len(fifa_file.columns)))

+-------------+--------------+----+---------+---------+----------+
|     HomeTeam|      AwayTeam|Year|HomeGoals|AwayGoals|TotalGoals|
+-------------+--------------+----+---------+---------+----------+
|       France|        Mexico|1930|        4|        1|         5|
|      Uruguay|     Argentina|1930|        4|        2|         6|
|      Uruguay|    Yugoslavia|1930|        6|        1|         7|
|    Argentina| United States|1930|        6|        1|         7|
|     Paraguay|       Belgium|1930|        1|        0|         1|
|United States|      Paraguay|1930|        3|        0|         3|
|      Uruguay|       Romania|1930|        4|        0|         4|
|      Uruguay|          Peru|1930|        1|        0|         1|
|      Romania|          Peru|1930|        3|        1|         4|
|United States|       Belgium|1930|        3|        0|         3|
|   Yugoslavia|       Bolivia|1930|        4|        0|         4|
|   Yugoslavia|        Brazil|1930|        2|        1|       

In [4]:
# Filter the data for the home team. If the HomeTeam has exactly more goals than the AwayTeam, the HomeTeam has won
#Then we can add the game to the home_wins dataframe
home_wins = fifa_file.filter(fifa_file["HomeGoals"] > fifa_file["AwayGoals"])
#Show the data to test if it is correct
home_wins.show()
#Show the amount of rows to test if it is correct
print((home_wins.count(), len(home_wins.columns)))

+-------------+--------------+----+---------+---------+----------+
|     HomeTeam|      AwayTeam|Year|HomeGoals|AwayGoals|TotalGoals|
+-------------+--------------+----+---------+---------+----------+
|       France|        Mexico|1930|        4|        1|         5|
|      Uruguay|     Argentina|1930|        4|        2|         6|
|      Uruguay|    Yugoslavia|1930|        6|        1|         7|
|    Argentina| United States|1930|        6|        1|         7|
|     Paraguay|       Belgium|1930|        1|        0|         1|
|United States|      Paraguay|1930|        3|        0|         3|
|      Uruguay|       Romania|1930|        4|        0|         4|
|      Uruguay|          Peru|1930|        1|        0|         1|
|      Romania|          Peru|1930|        3|        1|         4|
|United States|       Belgium|1930|        3|        0|         3|
|   Yugoslavia|       Bolivia|1930|        4|        0|         4|
|   Yugoslavia|        Brazil|1930|        2|        1|       

In [5]:
# Now that we have all the winning home teams, we can group them by their name and count the wins
home_wins = home_wins.groupBy("HomeTeam").count()
#show the data to test if it is correct
home_wins.show()
#show the amount of rows to test if it is correct
print((home_wins.count(), len(home_wins.columns)))

+--------------+-----+
|      HomeTeam|count|
+--------------+-----+
|      Paraguay|    4|
|        Russia|    4|
|        Sweden|   16|
|        Turkey|    2|
|       Germany|   23|
|   Ivory Coast|    2|
|        France|   22|
|        Greece|    2|
|       Algeria|    1|
|      Slovakia|    1|
|     Argentina|   39|
|         Wales|    1|
|       Belgium|   14|
|       Ecuador|    2|
|         Ghana|    1|
|          Peru|    4|
| United States|    6|
|         Chile|    9|
|  Soviet Union|   13|
|Czechoslovakia|    9|
+--------------+-----+
only showing top 20 rows

(57, 2)


In [6]:
#Now we have to sort the data in descending order, so that the team with the most wins is at the top
home_wins = home_wins.sort("count", ascending=False)

#Print the first 3 entries of the dataframe
print("The team with the most home wins is: ")
home_wins.show()

#show the amount of rows to test if it is correct
print((home_wins.count(), len(home_wins.columns)))

The team with the most home wins is: 
+--------------+-----+
|      HomeTeam|count|
+--------------+-----+
|        Brazil|   54|
|     Argentina|   39|
|         Italy|   36|
|  West Germany|   27|
|       Germany|   23|
|        France|   22|
|       England|   21|
|       Uruguay|   18|
|         Spain|   16|
|        Sweden|   16|
|       Hungary|   15|
|   Netherlands|   15|
|       Belgium|   14|
|  Soviet Union|   13|
|      Portugal|   13|
|    Yugoslavia|   12|
|        Poland|   10|
|       Austria|   10|
|         Chile|    9|
|Czechoslovakia|    9|
+--------------+-----+
only showing top 20 rows
(57, 2)


In [7]:
#The dataset is from 2022, so we can filter the data for the last 10 years by filtering for the year > 2011
mostsuccessful = fifa_file.filter(fifa_file["Year"] > 2011)
#Delete all the games which ended in a draw
mostsuccessful = mostsuccessful.filter(mostsuccessful["HomeGoals"] != mostsuccessful["AwayGoals"])

In [8]:
#show the earliest and latest data from the dataset
mostsuccessful.select("Year").distinct().show()

+----+
|Year|
+----+
|2014|
|2018|
+----+


In [9]:
#Now we need a new columns to determine the winner of the game
#We can use the when function to create a new column with the winner of the game
from pyspark.sql.functions import when
mostsuccessful = mostsuccessful.withColumn("Winner", when(mostsuccessful["HomeGoals"] > mostsuccessful["AwayGoals"], mostsuccessful["HomeTeam"]).otherwise(mostsuccessful["AwayTeam"]))
#show the data to test if it is correct
mostsuccessful.show()
#Show the amount of rows to test if it is correct
print((mostsuccessful.count(), len(mostsuccessful.columns)))

+--------------------+--------------------+----+---------+---------+----------+--------------------+
|            HomeTeam|            AwayTeam|Year|HomeGoals|AwayGoals|TotalGoals|              Winner|
+--------------------+--------------------+----+---------+---------+----------+--------------------+
|       United States|             Germany|2014|        0|        1|         1|             Germany|
|             Belgium|              Russia|2014|        1|        0|         1|             Belgium|
|             Belgium|             Algeria|2014|        2|        1|         3|             Belgium|
|            Portugal|               Ghana|2014|        2|        1|         3|            Portugal|
|           Argentina|         Switzerland|2014|        1|        0|         1|           Argentina|
|               Ghana|       United States|2014|        1|        2|         3|       United States|
|             Germany|            Portugal|2014|        4|        0|         4|            

In [10]:
#Now we can group the data by the winner and count the wins
mostsuccessful = mostsuccessful.groupBy("Winner").count()

#Sort the data in descending order
mostsuccessful = mostsuccessful.sort("count", ascending=False)

#show the data to test if it is correct
mostsuccessful.show()

+-----------+-----+
|     Winner|count|
+-----------+-----+
|    Belgium|   10|
|     France|    9|
|    Germany|    7|
|  Argentina|    6|
|    Uruguay|    6|
|     Brazil|    6|
|   Colombia|    6|
|    Croatia|    5|
|Netherlands|    5|
|     Mexico|    4|
|     Sweden|    3|
|Switzerland|    3|
|    England|    3|
|     Russia|    2|
|      Chile|    2|
|    Nigeria|    2|
|      Spain|    2|
|   Portugal|    2|
| Costa Rica|    2|
|    Senegal|    1|
+-----------+-----+


In [11]:
"""
The next task is to find out which team/nation has scored the most goals of all time
"""

#First we need to create a new dataframe with the teams and the goals they scored
#We can do this by selecting the columns HomeTeam and HomeGoals and renaming them to Team and Goals
mostgoals = fifa_file.select("HomeTeam", "HomeGoals").withColumnRenamed("HomeTeam", "Team").withColumnRenamed("HomeGoals", "Goals")
#show the data to test if it is correct
mostgoals.show()
#show the amount of rows to test if it is correct
print((mostgoals.count(), len(mostgoals.columns)))

+-------------+-----+
|         Team|Goals|
+-------------+-----+
|       France|    4|
|      Uruguay|    4|
|      Uruguay|    6|
|    Argentina|    6|
|     Paraguay|    1|
|United States|    3|
|      Uruguay|    4|
|      Uruguay|    1|
|      Romania|    3|
|United States|    3|
|   Yugoslavia|    4|
|   Yugoslavia|    2|
|    Argentina|    3|
|    Argentina|    6|
|        Chile|    1|
|        Chile|    3|
|    Argentina|    1|
|       Brazil|    4|
|      Germany|    2|
|        Italy|    2|
+-------------+-----+
only showing top 20 rows

(900, 2)


In [12]:
#Now we add the away teams and the goals they scored to the dataframe
#We can do this by selecting the columns AwayTeam and AwayGoals and renaming them to Team and Goals
mostgoals = mostgoals.union(fifa_file.select("AwayTeam", "AwayGoals").withColumnRenamed("AwayTeam", "Team").withColumnRenamed("AwayGoals", "Goals"))
#show the data to test if it is correct
mostgoals.show()
#show the amount of rows to test if it is correct
print((mostgoals.count(), len(mostgoals.columns)))

+-------------+-----+
|         Team|Goals|
+-------------+-----+
|       France|    4|
|      Uruguay|    4|
|      Uruguay|    6|
|    Argentina|    6|
|     Paraguay|    1|
|United States|    3|
|      Uruguay|    4|
|      Uruguay|    1|
|      Romania|    3|
|United States|    3|
|   Yugoslavia|    4|
|   Yugoslavia|    2|
|    Argentina|    3|
|    Argentina|    6|
|        Chile|    1|
|        Chile|    3|
|    Argentina|    1|
|       Brazil|    4|
|      Germany|    2|
|        Italy|    2|
+-------------+-----+
only showing top 20 rows

(1800, 2)


In [13]:
#Convert the goals column to an integer
from pyspark.sql.types import IntegerType
mostgoals = mostgoals.withColumn("Goals", mostgoals["Goals"].cast(IntegerType()))
#Now we can group the data by the team and sum the goals
mostgoals = mostgoals.groupBy("Team").sum("Goals")
#show the data to test if it is correct
mostgoals.show()
#show the amount of rows to test if it is correct
print((mostgoals.count(), len(mostgoals.columns)))

+-----------+----------+
|       Team|sum(Goals)|
+-----------+----------+
|   Paraguay|        30|
|     Russia|        24|
|    Senegal|        11|
|     Sweden|        80|
|     Turkey|        20|
|      Zaire|         0|
|       Iraq|         1|
|    Germany|        95|
|Ivory Coast|        13|
|     France|       120|
|     Greece|         5|
|    Algeria|        13|
|       Togo|         1|
|   Slovakia|         5|
|  Argentina|       137|
|      Wales|         4|
|    Belgium|        68|
|     Angola|         1|
|    Ecuador|        10|
|      Ghana|        13|
+-----------+----------+
(85, 2)


In [14]:
#Sort the data in descending order
mostgoals = mostgoals.sort("sum(Goals)", ascending=False)
#show the data to test if it is correct
mostgoals.show()
#show the amount of rows to test if it is correct
print((mostgoals.count(), len(mostgoals.columns)))

+--------------+----------+
|          Team|sum(Goals)|
+--------------+----------+
|        Brazil|       229|
|     Argentina|       137|
|  West Germany|       131|
|         Italy|       128|
|        France|       120|
|         Spain|        99|
|       Germany|        95|
|       England|        91|
|       Uruguay|        87|
|       Hungary|        87|
|   Netherlands|        86|
|        Sweden|        80|
|       Belgium|        68|
|        Mexico|        60|
|    Yugoslavia|        55|
|  Soviet Union|        53|
|   Switzerland|        50|
|      Portugal|        49|
|        Poland|        46|
|Czechoslovakia|        44|
+--------------+----------+
only showing top 20 rows

(85, 2)
